# 🧪  Recolha e Limpeza

In [1]:
# Importar bibliotecas necessárias
import pandas as pd
import os
import requests
import holidays

In [2]:
df = pd.read_parquet('../datasets/consumos_horario_codigo_postal.parquet') # Abrir o dataset

In [3]:
df.dtypes  # Verificar tipos de dados

datahora         datetime64[ms, Europe/Lisbon]
dt_consumo                              object
hr_consumo                              object
codigo_postal                           object
consumo                                float64
dia_semana                              object
dtype: object

# Engenharia de features
| Feature   | Para que serve?                                   |
|-----------|---------------------------------------------------|
| Day       | Captura ciclos diários                            |
| Month     | Captura sazonalidade mensal                       |
| IsWeekend | 1 se for sábado ou domingo, senão 0               |
| TimeOfDay | Útil porque o consumo e diferente ao longo do dia |
| Season    | 	Captura efeitos climáticos ou sazonais amplos    |

In [4]:
df['DateTime'] = pd.to_datetime(df['dt_consumo'].astype(str) + ' ' + df['hr_consumo'], format='%Y-%m-%d %H:%M') #  Criar a coluna DateTime

df['Date'] = df['dt_consumo'] #  Criar a coluna Date

df['Hour'] = pd.to_datetime(df['hr_consumo'], format='%H:%M').dt.hour # Extrair a hora como número inteiro (ex: 23 de '23:00')

df['ZipCode'] = df['codigo_postal'] #  Criar a coluna ZipCode

df['ActiveEnergy(kWh)'] = df['consumo'] #  Criar a coluna ActiveEnergy(kWh)

df['Day'] = df['DateTime'].dt.day #  Criar a coluna Day

df['Month'] = df['DateTime'].dt.month #  Criar a coluna Month

df['Year'] = df['DateTime'].dt.year #  Criar a coluna Year

df['IsWeekend'] = df['dia_semana'].isin(['Sábado', 'Domingo']).astype(int) #  Criar a coluna IsWeekend

#  Criar a coluna TimeOfDay
def classificar_periodo(hora):
    if 0 <= hora <= 6:
        return 'Noite'
    elif 7 <= hora <= 11:
        return 'Manhã'
    elif 12 <= hora <= 18:
        return 'Tarde'
    else:
        return 'Noite'

df['TimeOfDay'] = df['Hour'].apply(classificar_periodo)

df['DayOfTheWeek'] = df['dia_semana'] #  Criar a coluna DayOfTheWeek

#  Criar a coluna Season
def get_season(month):
    if month in [12, 1, 2]:
        return 'Inverno'
    elif month in [3, 4, 5]:
        return 'Primavera'
    elif month in [6, 7, 8]:
        return 'Verão'
    else:
        return 'Outono'

df['Season'] = df['Month'].apply(get_season)

# Criar a coluna IsHoliday
feriados_pt = holidays.Portugal(years=[2022, 2023])
df['IsHoliday'] = df['DateTime'].dt.date.isin(feriados_pt).astype(int)

df.drop(columns=['datahora', 'dt_consumo', 'hr_consumo', 'codigo_postal', 'consumo', 'dia_semana'], inplace=True) # Eliminar a coluna Date/Time

df.tail() # Mostra as ultimas 5 linhas do DataFrame df_total (útil para inspeção rápida dos dados)

,DateTime,Date,Hour,ZipCode,ActiveEnergy(kWh),Day,Month,Year,IsWeekend,TimeOfDay,DayOfTheWeek,Season,IsHoliday
3727434,2023-02-09 10:00:00,2023-02-09,10,4405,19433.670144,9,2,2023,0,Manhã,Quinta,Inverno,0
3727435,2023-02-14 10:00:00,2023-02-14,10,4870,2314.309648,14,2,2023,0,Manhã,Terça,Inverno,0
3727436,2023-02-07 15:00:00,2023-02-07,15,4475,113968.770238,7,2,2023,0,Tarde,Terça,Inverno,0
3727437,2023-02-23 21:00:00,2023-02-23,21,5350,1731.337009,23,2,2023,0,Noite,Quinta,Inverno,0
3727438,2023-02-19 12:00:00,2023-02-19,12,3610,2519.575060,19,2,2023,1,Tarde,Domingo,Inverno,0


In [5]:
# Verifica valores ausentes ou duplicados
df.isnull().sum()
df.duplicated().sum()

0

In [6]:
df = df.sort_values(by='DateTime').reset_index(drop=True) # ordenar os dados pela coluna Datetime

# Adicionar ao dataset, APIs referentes a temperatura, a densidade populacional e ao feriado

In [7]:
caminho_da_pasta = '../datasets/datasetsPorCP'

# Dividir o dataset por varios datasets, por codigo postal
for cp in df['ZipCode'].unique(): # Para cada código postal único no dataset
    if cp == "OUTROS":
        print(f"⚠️ A ignorar código postal inválido: {cp}")
        continue
    df_cp = df[df['ZipCode'] == cp].copy()
    df_cp.sort_values('DateTime', inplace=True)  # Ordenar por DateTime (por segurança)

    if not os.path.exists(caminho_da_pasta): # Verifique se a pasta já existe
        os.mkdir(caminho_da_pasta) # Crie a pasta
    caminho_ficheiro = f"{caminho_da_pasta}/consumo_{cp}.parquet"
    df_cp.to_parquet(caminho_ficheiro, index=False)  # Guardar como ficheiro parquet
    print(f"Ficheiro guardado: {caminho_ficheiro} ({len(df_cp)} registos)")

Ficheiro guardado: ../datasets/datasetsPorCP/consumo_5000.parquet (8016 registos)
Ficheiro guardado: ../datasets/datasetsPorCP/consumo_2450.parquet (8016 registos)
Ficheiro guardado: ../datasets/datasetsPorCP/consumo_5230.parquet (8016 registos)
Ficheiro guardado: ../datasets/datasetsPorCP/consumo_3620.parquet (8016 registos)
Ficheiro guardado: ../datasets/datasetsPorCP/consumo_2800.parquet (8016 registos)
Ficheiro guardado: ../datasets/datasetsPorCP/consumo_4780.parquet (8016 registos)
Ficheiro guardado: ../datasets/datasetsPorCP/consumo_2410.parquet (8016 registos)
Ficheiro guardado: ../datasets/datasetsPorCP/consumo_4585.parquet (8016 registos)
Ficheiro guardado: ../datasets/datasetsPorCP/consumo_2380.parquet (8016 registos)
Ficheiro guardado: ../datasets/datasetsPorCP/consumo_2985.parquet (8016 registos)
Ficheiro guardado: ../datasets/datasetsPorCP/consumo_5350.parquet (8016 registos)
Ficheiro guardado: ../datasets/datasetsPorCP/consumo_2480.parquet (8016 registos)
Ficheiro guardad

In [8]:
ficheiros = [f for f in os.listdir(caminho_da_pasta) if f.endswith(".parquet")] # Listar todos os ficheiros .parquet da pasta

# Guardar coordenadas em cache para não pedir várias vezes
coordenadas_por_distrito = {}
coordenadasConcelho_cache = {}

# Mapeamento simples de prefixos de CPs para distritos
prefixo_para_distrito = {
    range(1000, 2000): "Lisboa",
    range(2000, 2100): "Santarém",
    range(2100, 2300): "Lisboa",
    range(2300, 2400): "Santarém",
    range(2400, 2500): "Leiria",
    range(2500, 2700): "Lisboa",
    range(2700, 2800): "Lisboa",
    range(2800, 3000): "Setúbal",
    range(3000, 3100): "Coimbra",
    range(3100, 3300): "Leiria",
    range(3300, 3500): "Castelo Branco",
    range(3500, 3600): "Viseu",
    range(3600, 3700): "Viseu",
    range(3700, 3800): "Aveiro",
    range(3800, 3900): "Aveiro",
    range(3900, 4000): "Viseu",
    range(4000, 4200): "Porto",
    range(4200, 4400): "Porto",
    range(4400, 4500): "Vila Nova de Gaia",
    range(4500, 4600): "Penafiel",
    range(4600, 4700): "Amarante",
    range(4700, 4900): "Braga",
    range(4900, 5000): "Viana do Castelo",
    range(5000, 5100): "Vila Real",
    range(5100, 5400): "Bragança",
    range(5400, 5500): "Chaves",
    range(5500, 6000): "Bragança",
    range(6000, 6100): "Castelo Branco",
    range(6100, 6200): "Castelo Branco",
    range(6200, 6300): "Covilhã",
    range(6300, 6400): "Guarda",
    range(6400, 6500): "Guarda",
    range(7000, 7100): "Évora",
    range(7100, 7200): "Évora",
    range(7200, 7300): "Évora",
    range(7300, 7400): "Portalegre",
    range(7400, 7500): "Portalegre",
    range(7500, 7600): "Setúbal",
    range(7600, 7700): "Beja",
    range(7700, 7800): "Beja",
    range(7800, 7900): "Beja",
    range(7900, 8000): "Beja",
    range(8000, 8100): "Faro",
    range(8100, 8200): "Faro",
    range(8200, 8300): "Faro",
    range(8300, 8400): "Faro",
    range(8400, 8500): "Faro",
    range(8500, 8600): "Faro",
    range(8600, 8700): "Faro",
    range(8700, 8800): "Faro",
    range(8800, 8900): "Faro",
    range(8900, 9000): "Faro",
    range(9000, 9100): "Madeira",
    range(9100, 9200): "Madeira",
    range(9500, 9600): "Açores",
    range(9600, 9700): "Açores",
    range(9700, 9800): "Açores",
}

# Abrir o dataset referente a densidade populacional
df_densidadePopulacional = pd.read_csv('../datasets/ine_densidadePopulacional.csv', sep=";", skiprows=1, header=None, names=["RawConcelho", "Densidade"], encoding='utf-8')
df_densidadePopulacional[["ID_Concelho", "Concelho"]] = df_densidadePopulacional["RawConcelho"].str.split(":", expand=True) # Separa "1601:Arcos de Valdevez" em duas colunas
df_densidadePopulacional.drop(columns="RawConcelho", inplace=True) # Remove a coluna original
df_densidadePopulacional["Densidade"] = df_densidadePopulacional["Densidade"].str.replace(",", ".", regex=False).astype(float) # Corrige vírgula decimal

In [9]:
# Função auxiliar para obter distrito a partir do código postal
def obter_distrito(codigoPostal):
    try:
        prefixo = int(str(codigoPostal)[:4])
    except ValueError:
        return None  # Código postal inválido, como "OUTROS"

    for intervalo, distrito in prefixo_para_distrito.items():
        if prefixo in intervalo:
            return distrito
    return None  # Prefixo não mapeado

In [10]:
# Função para obter as coordenadas e concelho a partir do codigo postal
def obter_coordenadas_e_concelho(codigoPostal):
 # Ignora CPs que não pertençam a nenhum distrito conhecido
    # Ignora casos inválidos como "OUTROS"
    if not str(codigoPostal).isdigit():
        print(f"⚠️ A ignorar código postal inválido: {codigoPostal}")
        return None, None, None

    distrito = obter_distrito(codigoPostal)     # Verifica se já existem coordenadas em cache por distrito

    if distrito is None:
        print(f"⚠️ Código postal {codigoPostal} não está associado a nenhum distrito conhecido.")
        return None, None, None

    if distrito in coordenadas_por_distrito:
        return coordenadas_por_distrito[distrito]

    if codigoPostal in coordenadasConcelho_cache:
        return coordenadasConcelho_cache[codigoPostal]

    # Tenta obter coordenadas usando um CP representativo do distrito
    sufixos = [f"{i:03d}" for i in range(1, 1000, 2)]
    for sufixo in sufixos:
        url = f"https://www.cttcodigopostal.pt/api/v1/3d3ee9c11bb94d1ab881ff6133f48257/{codigoPostal}-{sufixo}"
        try:
            response = requests.get(url, timeout=5)
            response.raise_for_status()
            data = response.json()
            if data and isinstance(data, list):
                lat = float(data[0]["latitude"])
                lon = float(data[0]["longitude"])
                concelho = str(data[0]["concelho"])

                coordenadas_por_distrito[distrito] = (lat, lon, concelho)
                return lat, lon, concelho
        except requests.exceptions.RequestException:
            continue
        except Exception as e:
            print(f"⚠️ Erro inesperado em {codigoPostal}-{sufixo}: {e}")
            break

    print(f"❌ Falha: {codigoPostal} ({distrito}) sem coordenadas válidas")
    return None, None, None

In [11]:
# Função para obter a temperatura horária a partir das coordenadas
def obter_temperatura(lat, lon):
    url = "https://archive-api.open-meteo.com/v1/archive"  # URL da API de ficheiro histórico da Open-Meteo

    # Parâmetros da requisição, incluindo latitude, longitude, datas e tipo de dado (temperatura a 2 metros de altura)
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": "2022-11-01", # Data de início do período
        "end_date": "2023-09-30", # Data de fim do período
        "hourly": "temperature_2m", # Dados horários de temperatura
        "timezone": "auto" # Fuso horário automático com base na localização
    }
    try:
        response = requests.get(url, params=params)  # Faz a requisição GET à API com os parâmetros definidos
        response.raise_for_status() # Lança uma exceção se a resposta indicar erro (ex: 404, 500, etc.)
        data = response.json() # Converte a resposta JSON num dicionário Python

        times = pd.to_datetime(data["hourly"]["time"]) # Converte a lista de datas/horas para objetos datetime do pandas
        temps = data["hourly"]["temperature_2m"] # Extrai a lista de temperaturas correspondentes
        df_temp = pd.DataFrame({"DateTime": times, "Temperature": temps}) # Cria um DataFrame com duas colunas: Data/Hora e Temperatura

        return df_temp # Retorna o DataFrame criado
    except Exception as e:
        print(f"❌ Erro ao obter temperaturas: {e}") # Em caso de erro (ex: ligação falhada, dados ausentes), imprime mensagem de erro

        return pd.DataFrame(columns=["DateTime", "Temperature"]) # Retorna um DataFrame vazio com as colunas esperadas

In [12]:
# Funcao para obter a densidade populacional do concelho
def obter_densidade_populacional(concelho):
    if concelho not in df_densidadePopulacional["Concelho"].values: # Verifica se o concelho existe na coluna "Concelho" do DataFrame df_densidadePopulacional
        print(f"⚠️ Não existe concelho '{concelho}' no dataset de densidade populacional") # Caso o concelho não exista, imprime um aviso e retorna None
        return None

    densidade = df_densidadePopulacional.loc[df_densidadePopulacional["Concelho"] == concelho, "Densidade"].values[0]  # Seleciona a densidade correspondente ao concelho dado

    return densidade # Retorna o valor da densidade populacional

In [13]:
# Na pasta datasetsPorCP, abrir cada dataset
for ficheiro in ficheiros:
    caminho_ficheiro = os.path.join(caminho_da_pasta, ficheiro) # Cria o caminho completo para o ficheiro atual
    df = pd.read_parquet(caminho_ficheiro) # Lê o ficheiro Parquet para um DataFrame
    df = df.sort_values("DateTime") # Ordena os dados pela coluna DateTime

    codigoPostal = str(df["ZipCode"].iloc[0]) # Obtém o código postal do primeiro registo (assumindo que todos os registos têm o mesmo CP)
    lat, lon, concelho = obter_coordenadas_e_concelho(codigoPostal) # Obtém latitude, longitude e concelho associados ao código postal

    # Se alguma coordenada ou concelho estiver em falta, ignora este ficheiro
    if lat is None or lon is None or concelho is None:
        print(f"⚠️ A saltar {codigoPostal} (sem coordenadas e/ou concelho)")
        continue

    df_temp = obter_temperatura(lat, lon) # Obtém as temperaturas horárias para as coordenadas dadas

    # Se não houver dados de temperatura, ignora este ficheiro
    if df_temp.empty:
        print(f"⚠️ Sem dados de temperatura para {codigoPostal}")
        continue

    # 🔄 Garante que ambas as colunas DateTime têm o mesmo formato (arredondado à hora)
    df["DateTime"] = pd.to_datetime(df["DateTime"]).dt.floor("h")
    df_temp["DateTime"] = pd.to_datetime(df_temp["DateTime"]).dt.floor("h")

    # 🔗 Junta os dados da temperatura ao DataFrame original com base na coluna DateTime
    df = pd.merge(df, df_temp, on="DateTime", how="left")

    # Verifica novamente o código postal (embora já tenha sido verificado antes)
    if codigoPostal is None :
        print(f"⚠️ A saltar {lat}, {lon} || {codigoPostal} (sem municipio)")
        continue

    # Adiciona a densidade populacional ao DataFrame, com base no concelho
    df["PopulationDensity"] = obter_densidade_populacional(concelho)

    # 💾 Guarda o DataFrame atualizado no mesmo ficheiro .parquet, sobrescrevendo-o
    df.to_parquet(caminho_ficheiro, index=False)
    print(f"✅ Ficheiro atualizado: {ficheiro}")

✅ Ficheiro atualizado: consumo_1000.parquet
✅ Ficheiro atualizado: consumo_1050.parquet
✅ Ficheiro atualizado: consumo_1070.parquet
✅ Ficheiro atualizado: consumo_1100.parquet
✅ Ficheiro atualizado: consumo_1150.parquet
✅ Ficheiro atualizado: consumo_1170.parquet
✅ Ficheiro atualizado: consumo_1200.parquet
✅ Ficheiro atualizado: consumo_1250.parquet
✅ Ficheiro atualizado: consumo_1300.parquet
✅ Ficheiro atualizado: consumo_1350.parquet
✅ Ficheiro atualizado: consumo_1400.parquet
✅ Ficheiro atualizado: consumo_1495.parquet
✅ Ficheiro atualizado: consumo_1500.parquet
✅ Ficheiro atualizado: consumo_1600.parquet
✅ Ficheiro atualizado: consumo_1675.parquet
✅ Ficheiro atualizado: consumo_1685.parquet
✅ Ficheiro atualizado: consumo_1700.parquet
✅ Ficheiro atualizado: consumo_1750.parquet
✅ Ficheiro atualizado: consumo_1800.parquet
✅ Ficheiro atualizado: consumo_1885.parquet
✅ Ficheiro atualizado: consumo_1900.parquet
✅ Ficheiro atualizado: consumo_1950.parquet
✅ Ficheiro atualizado: consumo_1

# Unir todos os datasets num so

In [14]:
ficheiros = [f for f in os.listdir(caminho_da_pasta) if f.endswith(".parquet")] # Lista todos os ficheiros .parquet na pasta

dataframes = [] # Lista para acumular os DataFrames

In [15]:
# Ler e adicionar cada ficheiro à lista
for ficheiro in ficheiros:
    caminho_ficheiro = os.path.join(caminho_da_pasta, ficheiro) # Cria o caminho completo para o ficheiro atual
    df = pd.read_parquet(caminho_ficheiro) # Lê o ficheiro .parquet e armazena os dados num DataFrame
    dataframes.append(df) # Adiciona o DataFrame à lista "dataframes"

    print(f"✅ Lido: {ficheiro} ({len(df)} linhas)") # Imprime uma mensagem indicando que o ficheiro foi lido com sucesso e quantas linhas contém

✅ Lido: consumo_1000.parquet (8016 linhas)
✅ Lido: consumo_1050.parquet (8016 linhas)
✅ Lido: consumo_1070.parquet (8016 linhas)
✅ Lido: consumo_1100.parquet (8016 linhas)
✅ Lido: consumo_1150.parquet (8016 linhas)
✅ Lido: consumo_1170.parquet (8016 linhas)
✅ Lido: consumo_1200.parquet (8016 linhas)
✅ Lido: consumo_1250.parquet (8016 linhas)
✅ Lido: consumo_1300.parquet (8016 linhas)
✅ Lido: consumo_1350.parquet (8016 linhas)
✅ Lido: consumo_1400.parquet (8016 linhas)
✅ Lido: consumo_1495.parquet (8016 linhas)
✅ Lido: consumo_1500.parquet (8016 linhas)
✅ Lido: consumo_1600.parquet (8016 linhas)
✅ Lido: consumo_1675.parquet (8016 linhas)
✅ Lido: consumo_1685.parquet (8016 linhas)
✅ Lido: consumo_1700.parquet (8016 linhas)
✅ Lido: consumo_1750.parquet (8016 linhas)
✅ Lido: consumo_1800.parquet (8016 linhas)
✅ Lido: consumo_1885.parquet (8016 linhas)
✅ Lido: consumo_1900.parquet (8016 linhas)
✅ Lido: consumo_1950.parquet (8016 linhas)
✅ Lido: consumo_1990.parquet (8016 linhas)
✅ Lido: con

In [16]:
df_total = pd.concat(dataframes, ignore_index=True) # Junta todos os DataFrames da lista 'dataframes' num único DataFrame

df_total["DateTime"] = pd.to_datetime(df_total["DateTime"])  # Converte a coluna "DateTime" para o tipo datetime, caso ainda não esteja nesse formato
df_total = df_total.sort_values("DateTime") # Ordena os dados pela coluna "DateTime" em ordem crescente

print(f"\n📦 Total combinado: {len(df_total)} linhas") # Imprime o número total de linhas após a concatenação


📦 Total combinado: 3719423 linhas


In [17]:
# Guardar versão limpa do dataset como .parquet

df_total = df_total[df_total['ActiveEnergy(kWh)'] >= 0] # Filtra o DataFrame para manter apenas os registos com energia ativa maior ou igual a 0 (remove valores negativos que podem indicar erros ou dados inválidos)
df_total.to_parquet("../datasets/consumo_eredes_customizado.parquet", index=False) # Guarda o DataFrame filtrado num ficheiro .parquet na pasta indicada, sem incluir o índice
print("✅ Ficheiro combinado guardado em: datasets/consumo_eredes_customizado.parquet") # Imprime uma mensagem a confirmar que o ficheiro foi guardado com sucesso

✅ Ficheiro combinado guardado em: datasets/consumo_eredes_customizado.parquet


In [18]:
# Apagar pasta "datasetsPorCP"

for root, dirs, files in os.walk(caminho_da_pasta, topdown=False): # Apagar todos os ficheiros e subpastas dentro da pasta. Usa os.walk() com topdown=False para começar pelas subpastas mais profundas e evitar erros ao apagar diretórios
    for name in files:
        os.remove(os.path.join(root, name)) # Apaga cada ficheiro encontrado
    for name in dirs:
        os.rmdir(os.path.join(root, name)) # Apaga cada subpasta encontrada

os.rmdir(caminho_da_pasta) # Após esvaziar o conteúdo, apaga a própria pasta principal

In [19]:
df_total.head() # Mostra as primeiras 5 linhas do DataFrame df_total (útil para inspeção rápida dos dados)

,DateTime,Date,Hour,ZipCode,ActiveEnergy(kWh),Day,Month,Year,IsWeekend,TimeOfDay,DayOfTheWeek,Season,IsHoliday,Temperature,PopulationDensity
0,2022-11-01,2022-11-01,0,1000,9328.306723,1,11,2022,0,Noite,Terça,Outono,1,15.5,5455.23
1186368,2022-11-01,2022-11-01,0,3045,4293.076725,1,11,2022,0,Noite,Terça,Outono,1,11.4,440.88
1194384,2022-11-01,2022-11-01,0,3050,6608.606545,1,11,2022,0,Noite,Terça,Outono,1,11.4,440.88
1202400,2022-11-01,2022-11-01,0,3060,16832.631994,1,11,2022,0,Noite,Terça,Outono,1,11.4,440.88
1210416,2022-11-01,2022-11-01,0,3070,5894.381217,1,11,2022,0,Noite,Terça,Outono,1,11.4,440.88


### Dataset Limpo
- Nome: `consumo_eredes_customizado.parquet`
- Colunas mantidas: `Hour`, `Active Energy (kWh)`, `Day of the Week`
- Colunas criadas: `DateTime`, `Date`, `Day`, `Month`, `Year`, `IsWeekend`, `TimeOfDay`, `Season`, `isHoliday`, `Temperature`, `PopulationDensity`
- `DateTime` ordenado cronologicamente